In [1]:
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn import preprocessing
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.pandas.set_option('display.max_rows', None)
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import time
from contextlib import contextmanager
import warnings
from lightgbm import LGBMClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.pandas.set_option('display.max_rows', None)
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)


@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))


# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category=True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns


def application_train_test(num_rows=None, nan_as_category=False):
    # Read data and merge
    df = pd.read_csv(r'C:\Users\oe\Desktop\HomeCreditDefaultRisk\data\application_train.csv', nrows=None)
    test_df = pd.read_csv(r'C:\Users\oe\Desktop\HomeCreditDefaultRisk\data\application_test.csv', nrows=None)
    print("Train samples: {}, test samples: {}".format(len(df), len(test_df)))
    df = df.append(test_df).reset_index()

    # DATA PREPROCESSING
    df = df[df['CODE_GENDER'] != 'XNA']  # 4 gözlem değeri XNA olarak girilmiş bundan kurtarıldı.
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)  # NaN değerleri 365243 olarak girilmiş, onlar düzeltildi
    df["OWN_CAR_AGE"] = df["OWN_CAR_AGE"].fillna(0)  # Araba yaş değeri boş olan gözlemler 0 olarak atandı

    # FEATURE ENGINEERING
    # AGE gün cinsinden belirtilmiş bu normal yaşa çevrildi.
    df["NEW_AGE"] = round(-1 * (df["DAYS_BIRTH"] / 365), 0)
    df["NEW_AGE"] = df["NEW_AGE"].astype("int")

    # NEW FEATURES
    df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']  # Müşterinin çalıştığı gün sayısının yaşına oranı
    df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df[
        'AMT_CREDIT']  # Müşterinin yıllık toplam gelirinin kredi miktarına oranı
    df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']  # Kredinin yıllık ödemesinin, kredinin tamamına oranı

    # FEATURE 1 - MAAŞ / AİLEDEKİ KİŞİ SAYISI
    df['NEW_INC_PERS'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    # FEATURE 2 - KREDİ MİKTARI / AİLEDEKİ KİŞİ SAYISI
    df['NEW_AMT/FAM'] = df['AMT_CREDIT'] / df['CNT_FAM_MEMBERS']
    # FEATURE 3 - KREDİNİN YILLIK ÖDEMESİ / GELİR
    df['NEW_ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    # FEATURE 4 - GELİR / YILLIK KREDİ * FAMILYSIZE #MODEL SONUCUNA GÖRE DEĞERLENDİR
    df["NEW_FAMILY_EFFECT"] = df['NEW_AMT/FAM'] / df['CNT_FAM_MEMBERS']
    # FEATURE 6 - ÇEKİLEN KREDİ İLE ÜRÜN ARASINDAKİ FARKIN GELİRE ORANI ***
    df["NEW_C-GP"] = (df["AMT_GOODS_PRICE"] - df["AMT_CREDIT"]) / df["AMT_INCOME_TOTAL"]
    # FEATURE 7 - YAŞ / KREDİ MİKTARI
    df["NEW_CREDIT/NEW_AGE"] = df['AMT_CREDIT'] / df["NEW_AGE"]
    # FEATURE 8 - ÜRÜN / KREDİ MİKTARI ***
    df["NEW_GOODS/CREDIT"] = df["AMT_GOODS_PRICE"] / df["AMT_CREDIT"]
    # FEATURE 9 - AGE / OWN_CAR_AGE
    df["NEW_AGE/CAR_AGE"] = df["NEW_AGE"] / df["OWN_CAR_AGE"]
    # FEATURE 10 - EXT AĞIRLIKLI ÇARPIM
    df['NEW_EXT_WEIGHTED'] = df.EXT_SOURCE_1 * 2 + df.EXT_SOURCE_2 * 1 + df.EXT_SOURCE_3 * 3
    # FEATURE 11 - EXT MEAN
    df["NEW_EXT_MEAN"] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    # FEATURE 12 - EXT STD
    df['NEW_SCORES_STD'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
    df['NEW_SCORES_STD'] = df['NEW_SCORES_STD'].fillna(df['NEW_SCORES_STD'].mean())
    # FEATURE 13 - NEW EXT PROCESS
    # df.loc[(df["EXT_SOURCE_1"] >= 0.5) | (df["EXT_SOURCE_2"] >= 0.55) | (df["EXT_SOURCE_3"] >= 0.45), "NEW_BOMB"] = 0
    # df.loc[(df["EXT_SOURCE_1"] < 0.5) | (df["EXT_SOURCE_2"] < 0.55) | (df["EXT_SOURCE_3"] < 0.45), "NEW_BOMB"] = 1
    # FEATURE 14 - DOKUMANLARIN TOPLAMI / DOCS ATILDI
    docs = [f for f in df.columns if 'FLAG_DOC' in f]
    df['NEW_DOCUMENT_COUNT'] = df[docs].sum(axis=1)
    df.drop(docs, axis=1, inplace=True)
    # FEATURE 15 - AGE RANK 1: YOUNG 5: OLDER
    # df["NEW_AGE_RANK"] = pd.cut(x=df["NEW_AGE"], bins=[0, 27, 40, 50, 65, 99], labels=[1, 2, 3, 4, 5])
    # df["NEW_AGE_RANK"] = df["NEW_AGE_RANK"].astype("int")
    # df.loc[(df["DAYS_BIRTH"] >= -15000),"NEW_YOUNG_FLAG"] = 1
    df.drop("NEW_AGE", axis=1, inplace=True)
    # FEATURE 16 NEW_PHONE_TO_BIRTH_RATIO
    df['NEW_PHONE_TO_BIRTH_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']
    # FEATURE 17 NEW_PHONE_TO_BIRTH_RATIO_EMPLOYER
    df['NEW_PHONE_TO_BIRTH_RATIO_EMPLOYER'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_EMPLOYED']
    # FEATURE 18 - NEW_INC_ORG : Sektöründeki maaş ortalamaları
    INC_ORG = df[['AMT_INCOME_TOTAL', 'ORGANIZATION_TYPE']].groupby('ORGANIZATION_TYPE').median()['AMT_INCOME_TOTAL']
    df['NEW_INC_ORG'] = df['ORGANIZATION_TYPE'].map(INC_ORG)
    # FEATURE 19 - NEW_PHO/ANNU : TELEFON / YILLIK ÖDEME
    df['NEW_PHO/ANNU'] = df['DAYS_LAST_PHONE_CHANGE'] / df['AMT_ANNUITY']
    # FEATURE 20 - NEW_PHO/ANNU : TELEFON / KAYIT DEĞİŞTİRME
    df["NEW_PHO/REG"] = df['DAYS_LAST_PHONE_CHANGE'] * df["DAYS_REGISTRATION"]
    # FEATURE 20 - NEW_PHO/ANNU : ADRES UYUMSUZLUKLARI
    df["NEW_FRAUD_1"] = df["REG_CITY_NOT_LIVE_CITY"] + df["REG_CITY_NOT_WORK_CITY"] + df["LIVE_CITY_NOT_WORK_CITY"]
    df["NEW_FRAUD"] = (df["NEW_FRAUD_1"] + 1) * df["DAYS_ID_PUBLISH"]
    del df["NEW_FRAUD_1"]
    df["NEW_FRAUD_std"] = (df[["REG_CITY_NOT_LIVE_CITY", "REG_CITY_NOT_WORK_CITY", "LIVE_CITY_NOT_WORK_CITY"]]).std(axis=1)


    # CLEAN CLASSES & LABEL ENCODING PART

    df.loc[(df["OCCUPATION_TYPE"] == "Drivers"), "OCCUPATION_TYPE"] = 1
    df.loc[(df["OCCUPATION_TYPE"] == "Waiters/barmen staff"), "OCCUPATION_TYPE"] = 1
    df.loc[(df["OCCUPATION_TYPE"] == "Low-skill Laborers"), "OCCUPATION_TYPE"] = 1
    df.loc[(df["OCCUPATION_TYPE"] == "Cleaning staff"), "OCCUPATION_TYPE"] = 2
    df.loc[(df["OCCUPATION_TYPE"] == "Sales staff"), "OCCUPATION_TYPE"] = 2
    df.loc[(df["OCCUPATION_TYPE"] == "Laborers"), "OCCUPATION_TYPE"] = 2
    df.loc[(df["OCCUPATION_TYPE"] == "Security staff"), "OCCUPATION_TYPE"] = 2
    df.loc[(df["OCCUPATION_TYPE"] == "Cooking staff"), "OCCUPATION_TYPE"] = 2
    df.loc[(df["OCCUPATION_TYPE"] == "Medicine staff"), "OCCUPATION_TYPE"] = 3
    df.loc[(df["OCCUPATION_TYPE"] == "Private service staff"), "OCCUPATION_TYPE"] = 3
    df.loc[(df["OCCUPATION_TYPE"] == "Realty agents"), "OCCUPATION_TYPE"] = 3
    df.loc[(df["OCCUPATION_TYPE"] == "Secretaries"), "OCCUPATION_TYPE"] = 3
    df.loc[(df["OCCUPATION_TYPE"] == "Accountants"), "OCCUPATION_TYPE"] = 4
    df.loc[(df["OCCUPATION_TYPE"] == "Core staff"), "OCCUPATION_TYPE"] = 4
    df.loc[(df["OCCUPATION_TYPE"] == "HR staff"), "OCCUPATION_TYPE"] = 4
    df.loc[(df["OCCUPATION_TYPE"] == "High skill tech staff"), "OCCUPATION_TYPE"] = 4
    df.loc[(df["OCCUPATION_TYPE"] == "Managers"), "OCCUPATION_TYPE"] = 4
    df.loc[(df["OCCUPATION_TYPE"] == "Medicine staff"), "OCCUPATION_TYPE"] = 4
    df.loc[(df["OCCUPATION_TYPE"] == "Private service staff"), "OCCUPATION_TYPE"] = 4
    df.loc[(df["OCCUPATION_TYPE"] == "Realty agents"), "OCCUPATION_TYPE"] = 4
    df.loc[(df["OCCUPATION_TYPE"] == "Secretaries"), "OCCUPATION_TYPE"] = 4
    df.loc[(df["OCCUPATION_TYPE"] == "IT staff"), "OCCUPATION_TYPE"] = 4


    def cols(dataframe, target, noc=10, ID=True):
        """
        noc : number of classes to threshold
        ID : if your data has ID, index etc
        """
        vars_more_classes = []
        if ID:
            ID = dataframe.columns[0]
        else:
            ID = "x"

        cat_cols = [col for col in dataframe.columns if dataframe[col].nunique() < noc
                    and col not in target]

        num_cols = [col for col in dataframe.columns if dataframe[col].nunique() > noc
                    and dataframe[col].dtypes != "O"
                    and col not in target
                    and col not in cat_cols and col not in ID]

        other_cols = [col for col in dataframe.columns if col not in cat_cols
                      and col not in num_cols and col not in ID
                      and col not in target]
        return cat_cols, num_cols, other_cols


    drop_list = [
        'FLAG_EMP_PHONE', 'FLAG_MOBIL', 'FLAG_CONT_MOBILE',
        'LIVE_REGION_NOT_WORK_REGION', 'FLAG_EMAIL', 'FLAG_PHONE',
        'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE',
        'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_WEEK',
        'COMMONAREA_MODE', 'FLOORSMAX_MODE', 'FLOORSMIN_MODE',
        'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 'NONLIVINGAPARTMENTS_MODE',
        'NONLIVINGAREA_MODE', 'ELEVATORS_MEDI', 'EMERGENCYSTATE_MODE',
        'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', "REG_REGION_NOT_LIVE_REGION",
        "LIVE_REGION_NOT_WORK_REGION",
        "AMT_REQ_CREDIT_BUREAU_HOUR", "HOUR_APPR_PROCESS_START"
    ]

    df.drop(drop_list, axis=1, inplace=True)

    cat_cols, num_cols, other_cols = cols(df, "TARGET")


    def rare_encoder(dataframe, rare_perc):
        temp_df = dataframe.copy()

        rare_columns = [col for col in temp_df.columns if temp_df[col].dtypes == 'O'
                        and (temp_df[col].value_counts() / len(temp_df) < rare_perc).any(axis=None)]

        for var in rare_columns:
            tmp = temp_df[var].value_counts() / len(temp_df)
            rare_labels = tmp[tmp < rare_perc].index
            temp_df[var] = np.where(temp_df[var].isin(rare_labels), 'Rare', temp_df[var])

        return temp_df


    def rare_analyser(dataframe, target, rare_perc):
        rare_columns = [col for col in dataframe.columns if
                        dataframe[col].dtypes == "O" and len(dataframe[col].value_counts()) <= 20
                        and (dataframe[col].value_counts() / len(dataframe) < rare_perc).any(axis=None)]

        for var in rare_columns:
            print(var, ":", len(dataframe[var].value_counts()))
            print(pd.DataFrame({"COUNT": dataframe[var].value_counts(),
                                "RATIO": dataframe[var].value_counts() / len(dataframe),
                                "TARGET_MEDIAN": dataframe.groupby(var)[target].median()}), end="\n\n\n")

    df = rare_encoder(df, 0.10)


    def label_encoder(dataframe, categorical_columns):
        """
        2 sınıflı kategorik değişkeni 0-1 yapma
        :param dataframe: İşlem yapılacak dataframe
        :param categorical_columns: Label encode yapılacak kategorik değişken adları
        :return:
        """
        labelencoder = preprocessing.LabelEncoder()

        for col in categorical_columns:
            if dataframe[col].nunique() == 2:
                dataframe[col] = labelencoder.fit_transform(dataframe[col])
        return dataframe


    df = label_encoder(df, cat_cols)

    df, app_cols = one_hot_encoder(df, True)


    return df

In [3]:
# Preprocess bureau.csv and bureau_balance.csv
def bureau_and_balance(num_rows = None, nan_as_category = True):
    bureau = pd.read_csv(r'C:\Users\oe\Desktop\HomeCreditDefaultRisk\data\bureau.csv', nrows = num_rows)
    bb = pd.read_csv(r'C:\Users\oe\Desktop\HomeCreditDefaultRisk\data\bureau_balance.csv', nrows = num_rows)
     
    # RARE ENCODING

    bureau.loc[(bureau["CREDIT_TYPE"] == "Microloan"),"CREDIT_TYPE"] = "Rare"
    bureau.loc[(bureau["CREDIT_TYPE"] == "Loan for business development"),"CREDIT_TYPE"] = "Rare"
    bureau.loc[(bureau["CREDIT_TYPE"] == "Another type of loan"),"CREDIT_TYPE"] = "Rare"
    bureau.loc[(bureau["CREDIT_TYPE"] == "Loan for working capital replenishment"),"CREDIT_TYPE"] = "Rare"
    bureau.loc[(bureau["CREDIT_TYPE"] == "Unknown type of loan"),"CREDIT_TYPE"] = "Rare"
    bureau.loc[(bureau["CREDIT_TYPE"] == "Cash loan (non-earmarked)"),"CREDIT_TYPE"] = "Rare"
    bureau.loc[(bureau["CREDIT_TYPE"] == "Real estate loan"),"CREDIT_TYPE"] = "Rare"
    bureau.loc[(bureau["CREDIT_TYPE"] == "Loan for the purchase of equipment"),"CREDIT_TYPE"] = "Rare"
    #credit active or not
    bureau.loc[(bureau["CREDIT_ACTIVE"] == "Bad debt"),"CREDIT_ACTIVE"] = "Active"
    bureau.loc[(bureau["CREDIT_ACTIVE"] == "Sold"),"CREDIT_ACTIVE"] = "Active"

    # GECİKMİŞ BORCU OLANLAR BORÇ MİKTARINA GÖRE ETİKETLENDİ
    bureau["AMT_CREDIT_MAX_OVERDUE"] = bureau["AMT_CREDIT_MAX_OVERDUE"].fillna(0) #nan değerler borcu yok diye değerlendirildi. model sonucuna göre kontrol edilecek.
    bureau.loc[((bureau["AMT_CREDIT_MAX_OVERDUE"] >= 0) | (bureau["AMT_CREDIT_SUM_OVERDUE"] >= 0)), "NEW_DANGER"] = 0 #gecikmiş borcu olmayanlar 
    bureau.loc[((bureau["AMT_CREDIT_MAX_OVERDUE"] >= 1) | (bureau["AMT_CREDIT_SUM_OVERDUE"] >= 1)), "NEW_DANGER"] = 1 #100.000'e kadar gecikmiş (başka) kredi borcu olanlar 
    bureau.loc[((bureau["AMT_CREDIT_MAX_OVERDUE"] >= 100000) | (bureau["AMT_CREDIT_SUM_OVERDUE"] >= 100000)), "NEW_DANGER"] = 2 #100.000'den 500000'e kadar gecikmiş (başka) kredi borcu olanlar 
    bureau.loc[((bureau["AMT_CREDIT_MAX_OVERDUE"] >= 500000) | (bureau["AMT_CREDIT_SUM_OVERDUE"] >= 500000)), "NEW_DANGER"] = 3 #500.000 den fazla gecikmiş borcu olup hayal kuranlar

    # "CNT_CREDIT_PROLONG" kredisi kaç kez uzatıldı? çeşitlilik yok.  Uzatma veya uzatmama durumuna indirgendi
    bureau.loc[(bureau["CNT_CREDIT_PROLONG"] == 0),"CNT_CREDIT_PROLONG"] = 0 #diğer yerlerdeki kredisini uzatmamış kişiler
    bureau.loc[(bureau["CNT_CREDIT_PROLONG"] != 0),"CNT_CREDIT_PROLONG"] = 1 #diğer yerlerdeki kredisini uzatmış kişiler

    # currency kategorilerdeki gözlem sayıları düşük olduğu için 1 ve 2 şeklinde kodlandı
    bureau.loc[(bureau["CREDIT_CURRENCY"] == "currency 1"),"CREDIT_CURRENCY"] = 0 #currency1
    bureau.loc[(bureau["CREDIT_CURRENCY"] == "currency 2"),"CREDIT_CURRENCY"] = 1 #currency2
    bureau.loc[(bureau["CREDIT_CURRENCY"] == "currency 3"),"CREDIT_CURRENCY"] = 1 #currency3
    bureau.loc[(bureau["CREDIT_CURRENCY"] == "currency 4"),"CREDIT_CURRENCY"] = 1 #currency4

    # FEATURE 1: BUREAU_LOAN_COUNT | Bir kişinin aldığı farklı kredi sayıları
    grp = bureau[['SK_ID_CURR', 'DAYS_CREDIT']].groupby(by = ['SK_ID_CURR'])['DAYS_CREDIT'].count().reset_index().rename(index=str, columns={'DAYS_CREDIT': 'NEW_BUREAU_LOAN_COUNT'})
    bureau = bureau.merge(grp, on = ['SK_ID_CURR'], how = 'left')

    # FEATURE 2: BUREAU_LOAN_TYPES | Bir kişinin kaç farklı tipte krediye sahip olduğu
    grp = bureau[['SK_ID_CURR', 'CREDIT_TYPE']].groupby(by = ['SK_ID_CURR'])['CREDIT_TYPE'].nunique().reset_index().rename(index=str, columns={'CREDIT_TYPE': 'NEW_BUREAU_LOAN_TYPES'})
    bureau = bureau.merge(grp, on = ['SK_ID_CURR'], how = 'left')

    # FEATURE 3: AVERAGE_LOAN_TYPE| Bir kişinin aldığı farklı kredi türlerinin oranı
    bureau['NEW_AVERAGE_LOAN_TYPE'] = bureau['NEW_BUREAU_LOAN_COUNT'] / bureau['NEW_BUREAU_LOAN_TYPES'] # bir kişinin aldığı farklı kredi tipi oranı. bir kişi hep aynı türden mi kredi almış, yoksa farklı türlerden mi?

    # FEATURE 4: AVERAGE_LOAN_TYPE | Aktif durumda olan kredilerin % si
    # Kredi aktif durumunu yeni bir değişkende gösteriyoruz. 0 kredileri kapanmış, 1 kredileri devam ediyor.  
    bureau.loc[(bureau['CREDIT_ACTIVE'] == "Closed"), 'CREDIT_ACTIVE_BINARY'] = 0
    bureau.loc[(bureau['CREDIT_ACTIVE'] != "Closed"), 'CREDIT_ACTIVE_BINARY'] = 1
    bureau['CREDIT_ACTIVE_BINARY'] = bureau['CREDIT_ACTIVE_BINARY'].astype('int32')

    # MÜŞTERİ başına AKTİF olan ortalama kredi sayısını hesaplayın
    grp = bureau.groupby(by = ['SK_ID_CURR'])['CREDIT_ACTIVE_BINARY'].mean().reset_index().rename(index=str, columns={'CREDIT_ACTIVE_BINARY': 'NEW_ACTIVE_LOANS_PERCENTAGE'}) #1e yakın olması kötü oluyor
    bureau = bureau.merge(grp, on = ['SK_ID_CURR'], how = 'left') #ana tablo ile birleştirme
    del bureau['CREDIT_ACTIVE_BINARY'] #gereksiz olan sütun atıldı
    gc.collect()

    # FEATURE 5: NEW_DAYS_DIFF | Bir kişi kaç gün aralıkla yeni krediler almış
    #Bir kişinin almış olduğu farklı kredileri alma günleri sıralandı
    grp = bureau[['SK_ID_CURR', 'SK_ID_BUREAU', 'DAYS_CREDIT']].groupby(by = ['SK_ID_CURR'])
    grp1 = grp.apply(lambda x: x.sort_values(['DAYS_CREDIT'], ascending = False)).reset_index(drop = True)#rename(index = str, columns = {'DAYS_CREDIT': 'DAYS_CREDIT_DIFF'})
    print("Grouping and Sorting done")
    grp1['DAYS_CREDIT1'] = grp1['DAYS_CREDIT']*-1 # günler - olarak getirildiğinden + yapıldı
    grp1['NEW_DAYS_DIFF'] = grp1.groupby(by = ['SK_ID_CURR'])['DAYS_CREDIT1'].diff() # aldığı farklı krediler arasında kaçar gün olduğu hesaplandı
    grp1['NEW_DAYS_DIFF'] = grp1['NEW_DAYS_DIFF'].fillna(0).astype('uint32') #ilk değişkende nan geleceği için 0 ile doldurdum. diff fonksiyonunda 2. değerden 1. değer çıkarılıyor. bu sebeple ilk değerde nan geliyor.
    del grp1['DAYS_CREDIT1'], grp1['DAYS_CREDIT'], grp1['SK_ID_CURR'] #gereksiz columnlar atıldı
    print("Difference days calculated")
    # ana tablo ile birleştirme işlemleri
    bureau = bureau.merge(grp1, on = ['SK_ID_BUREAU'], how = 'left')
    print("Difference in Dates between Previous CB applications is CALCULATED ")

    #FEATURE 6: NEW_CREDIT_ENDDATE_PERCENTAGE | ÖDEMESİ DEVAM EDEN KREDİ SAYISI ORTALAMASI
    bureau.loc[bureau['DAYS_CREDIT_ENDDATE'] < 0,"CREDIT_ENDDATE_BINARY"] = 0  #ödemesi bitmiş (Closed) krediler
    bureau.loc[bureau['DAYS_CREDIT_ENDDATE'] >= 0,"CREDIT_ENDDATE_BINARY"] = 1 #ödemesi devam eden (Active) krediler
    grp = bureau.groupby(by = ['SK_ID_CURR'])['CREDIT_ENDDATE_BINARY'].mean().reset_index().rename(index=str, columns={'CREDIT_ENDDATE_BINARY': 'NEW_CREDIT_ENDDATE_PERCENTAGE'}) #ödemesi devam eden kredi sayısının ortalamaları
    del bureau['CREDIT_ENDDATE_BINARY'] #gereksiz olan binary columnun düşürülmesi
    bureau = bureau.merge(grp, on = ['SK_ID_CURR'], how = 'left') #ana tabloya ekleme

    #FEATURE 7: NEW_AMT_PER_PAY | ÖDENEN BORÇ %Sİ
    bureau["NEW_AMT_PER_PAY"] = 1 - ((bureau["AMT_CREDIT_SUM"]- bureau["AMT_CREDIT_SUM_DEBT"]) / bureau["AMT_CREDIT_SUM"])

    #FEATURE 8: DAYS_ENDDATE_DIFF | Ödenmemiş krediler arasındaki gün farkları
    #NOT: Groupby aggregation işleminde mean ve sum alınabilir
    bureau['CREDIT_ENDDATE_BINARY'] = bureau['DAYS_CREDIT_ENDDATE']
    # Ödemesi devam edenler(+) ve ödemesi bitmiş olanlar(0 veya -) değerler belirtiliyor
    bureau.loc[(bureau["DAYS_CREDIT_ENDDATE"] <= 0), "CREDIT_ENDDATE_BINARY"] = 0 #ödemesi bitmiş (Closed) krediler
    bureau.loc[(bureau["DAYS_CREDIT_ENDDATE"] > 0), "CREDIT_ENDDATE_BINARY"] = 1 #ödemesi devam eden (Active) krediler
    #Ödemesi devam eden krediler üzerinde işlem yapılacak
    B1 = bureau[bureau['CREDIT_ENDDATE_BINARY'] == 1]
    del bureau["CREDIT_ENDDATE_BINARY"]

    #Ödemesi tamamlanmaya krediler arasındaki gün farklarının hesaplanması
    # Create Dummy Column for CREDIT_ENDDATE 
    B1['DAYS_CREDIT_ENDDATE1'] = B1['DAYS_CREDIT_ENDDATE']
    # Groupby Each Customer ID 
    grp = B1[['SK_ID_CURR', 'SK_ID_BUREAU', 'DAYS_CREDIT_ENDDATE1']].groupby(by = ['SK_ID_CURR'])
    # Kredi ödeme sürelerinin kişi özelinde küçükten büyüğe sıralanması
    grp1 = grp.apply(lambda x: x.sort_values(['DAYS_CREDIT_ENDDATE1'], ascending = True)).reset_index(drop = True) 
    del grp
    gc.collect()
    print("Grouping and Sorting done")
    # diff fonksiyonunda ilk satırlarda gelen nan valuler 0 ile dolduruldu.
    grp1['DAYS_ENDDATE_DIFF'] = grp1.groupby(by = ['SK_ID_CURR'])['DAYS_CREDIT_ENDDATE1'].diff()
    grp1['DAYS_ENDDATE_DIFF'] = grp1['DAYS_ENDDATE_DIFF'].fillna(0).astype('uint32')
    del grp1['DAYS_CREDIT_ENDDATE1'], grp1['SK_ID_CURR']
    gc.collect()
    print("Difference days calculated")

    # ana tablo ile birleştirilmesi
    bureau = bureau.merge(grp1, on = ['SK_ID_BUREAU'], how = 'left')
    del grp1, B1
    gc.collect()

    #FEATURE 9: Toplam Geciken Borç / Toplam Borç
    bureau['AMT_CREDIT_SUM_DEBT'] = bureau['AMT_CREDIT_SUM_DEBT'].fillna(0)  #nan değerler borç yok olarak alındı
    bureau['AMT_CREDIT_SUM_OVERDUE'] = bureau['AMT_CREDIT_SUM_OVERDUE'].fillna(0) #nan değerler gecikme yok olarak alındır
    #grp1 bir kişinin toplam borcu
    #grp2 bir kişinin toplam gecikmiş borcu
    grp1 = bureau[['SK_ID_CURR', 'AMT_CREDIT_SUM_DEBT']].groupby(by = ['SK_ID_CURR'])['AMT_CREDIT_SUM_DEBT'].sum().reset_index().rename( index = str, columns = { 'AMT_CREDIT_SUM_DEBT': 'TOTAL_CUSTOMER_DEBT'})
    grp2 = bureau[['SK_ID_CURR', 'AMT_CREDIT_SUM_OVERDUE']].groupby(by = ['SK_ID_CURR'])['AMT_CREDIT_SUM_OVERDUE'].sum().reset_index().rename( index = str, columns = { 'AMT_CREDIT_SUM_OVERDUE': 'TOTAL_CUSTOMER_OVERDUE'})
    #ana tabloya ekleme
    bureau = bureau.merge(grp1, on = ['SK_ID_CURR'], how = 'left') 
    bureau = bureau.merge(grp2, on = ['SK_ID_CURR'], how = 'left')
    del grp1, grp2
    gc.collect()
    bureau['NEW_OVERDUE_DEBT_RATIO'] = bureau['TOTAL_CUSTOMER_OVERDUE']/bureau['TOTAL_CUSTOMER_DEBT'] # kişinin toplam gecikmiş borcunun, toplam borcuna oranı
    del bureau['TOTAL_CUSTOMER_OVERDUE'], bureau['TOTAL_CUSTOMER_DEBT'] #gereksiz üretilen sütunlar kaldırıldı
    gc.collect()
    
    ### Bureau Balance 
    bb, bb_cat = one_hot_encoder(bb, nan_as_category)
    bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category)
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
    for col in bb_cat:
        bb_aggregations[col] = ['mean']
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
    bureau.drop(['SK_ID_BUREAU'], axis=1, inplace= True)
    del bb, bb_agg
    gc.collect()
    
    # Bureau and bureau_balance numeric features
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum']
    }
    # Bureau and bureau_balance categorical features
    cat_aggregations = {}
    for cat in bureau_cat: cat_aggregations[cat] = ['mean']
    for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']
    
    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
    # Bureau: Active credits - using only numerical aggregations
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')
    del active, active_agg
    gc.collect()
    # Bureau: Closed credits - using only numerical aggregations
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')
    del closed, closed_agg, bureau
    gc.collect()
    return bureau_agg

In [4]:
# Preprocess previous_applications.csv
def previous_applications(num_rows = None, nan_as_category = True):
    prev = pd.read_csv(r'C:\Users\oe\Desktop\HomeCreditDefaultRisk\data\previous_application.csv', nrows = num_rows)
    # Days 365.243 values -> nan
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
    #FEATURE ENGINEERING
    # ATILACAK  NFLAG_INSURED_ON_APPROVAL
    prev = prev.drop(["FLAG_LAST_APPL_PER_CONTRACT"],axis=1)
    prev = prev.drop(["NFLAG_LAST_APPL_IN_DAY"],axis=1)
    prev = prev.drop(["WEEKDAY_APPR_PROCESS_START"],axis=1)
    prev = prev.drop(["NAME_TYPE_SUITE"],axis=1)
    prev = prev.drop(["NFLAG_INSURED_ON_APPROVAL"],axis=1)
    prev = prev.drop(["NAME_SELLER_INDUSTRY"],axis=1)
    
    # AZ RASTLANAN DEĞİŞKENLER KENDİLERİNE YAKIN SINIFLARA AKTARILDILAR.
    prev.loc[(prev["NAME_PAYMENT_TYPE"] == "Cashless from the account of the employer"), "NAME_PAYMENT_TYPE"] = "Cash through the bank "
    # CODE_REJECT_REASON
    prev.loc[(prev["CODE_REJECT_REASON"] != "CLIENT") & (prev["CODE_REJECT_REASON"] != "XAP"), "CODE_REJECT_REASON"] = "HC"
    # CHANNEL_TYPE
    prev.loc[(prev["CHANNEL_TYPE"] == "Regional / Local"), "CHANNEL_TYPE"] = "Regional / Local / Stone"
    prev.loc[(prev["CHANNEL_TYPE"] == "Stone"), "CHANNEL_TYPE"] = "Regional / Local / Stone"   
    prev.loc[(prev["CHANNEL_TYPE"] == "Credit and cash offices"), "CHANNEL_TYPE"] = "Credit and cash offices / Channel"
    prev.loc[(prev["CHANNEL_TYPE"] == "Channel of corporate sales"), "CHANNEL_TYPE"] = "Credit and cash offices / Channel"
    # NAME_YIELD_GROUP
    prev.loc[(prev["NAME_YIELD_GROUP"] == "low_action"), "NAME_YIELD_GROUP" ] = "low"
    prev.loc[(prev["NAME_YIELD_GROUP"] == "low_normal"), "NAME_YIELD_GROUP"] = "low"
    prev.loc[(prev["NAME_YIELD_GROUP"] == "high"), "NAME_YIELD_GROUP"] = "mid/high"
    prev.loc[(prev["NAME_YIELD_GROUP"] == "middle"), "NAME_YIELD_GROUP"] = "mid/high"
    # NAME_PORTFOLIO
    prev.loc[(prev["NAME_PORTFOLIO"] == "Cards"), "NAME_PORTFOLIO"] = "Cards/Car/Cash"
    prev.loc[(prev["NAME_PORTFOLIO"] == "Cars"), "NAME_PORTFOLIO"] = "Cards/Car/Cash"
    prev.loc[(prev["NAME_PORTFOLIO"] == "Cash"), "NAME_PORTFOLIO"] = "Cards/Car/Cash"

    # Kabul edilme ve edilmeme durumuna göre ikiye ayrıldı ve verisetini değerlendirmek amacıyla target olarak kullanıldı.
    prev.loc[(prev["NAME_CONTRACT_STATUS"] == "Approved"),"NAME_CONTRACT_STATUS"] = 0
    prev.loc[(prev["NAME_CONTRACT_STATUS"] == "Unused offer"),"NAME_CONTRACT_STATUS"] = 0
    prev.loc[(prev["NAME_CONTRACT_STATUS"] == "Canceled"),"NAME_CONTRACT_STATUS"] = 1
    prev.loc[(prev["NAME_CONTRACT_STATUS"] == "Refused"),"NAME_CONTRACT_STATUS"] = 1
    prev["NAME_CONTRACT_STATUS"] = prev["NAME_CONTRACT_STATUS"].astype("int")
    
    #FEATURE 1 - NEW_APP_CREDIT_PERC | İstenen kredi ve aldığı kredi oranı
    prev['NEW_APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
    
    #FEATURE 2 - NEW_CREDIBILITY | müşterilerin cevap alma hızları & ONAY durumları
    prev["DAYS_DECISION2"] = prev["DAYS_DECISION"] * -1
    prev['ANS_SPEED'] = pd.cut(x = prev['DAYS_DECISION2'], bins = [0,100, 700, 3000], labels = ["Fast", "Normal", "Late"])
    del prev["DAYS_DECISION2"]

    prev.loc[((prev["NAME_CONTRACT_STATUS"] == "Approved" ) & (prev['ANS_SPEED'] == "Fast")),"NEW_CREDIBILITY"] = 5 #hızlı ve olumlu onay alanlar 
    prev.loc[((prev["NAME_CONTRACT_STATUS"] == "Approved" ) & (prev['ANS_SPEED'] == "Normal")),"NEW_CREDIBILITY"] = 4 #normal ve olumlu onay alanlar 
    prev.loc[((prev["NAME_CONTRACT_STATUS"] == "Approved" ) & (prev['ANS_SPEED'] == "Late")),"NEW_CREDIBILITY"] = 3 #yavaş ve olumlu onay alanlar 
    prev.loc[((prev["NAME_CONTRACT_STATUS"] == "Refused" ) & (prev['ANS_SPEED'] == "Late")),"NEW_CREDIBILITY"] = 2  #yavaş ve olumsuz onay alanlar 
    prev.loc[((prev["NAME_CONTRACT_STATUS"] == "Refused" ) & (prev['ANS_SPEED'] == "Normal")),"NEW_CREDIBILITY"] = 1  #normal ve olumsuz onay alanlar 
    prev.loc[((prev["NAME_CONTRACT_STATUS"] == "Refused" ) & (prev['ANS_SPEED'] == "Fast")),"NEW_CREDIBILITY"] = 0  #hızlı ve olumsuz onay alanlar 
    
    #FEATURE 3 - "NEW_ANN/CDT" | müşteri maaşının kredi tutarına oranı
    prev["NEW_ANN/CDT_PERC"] = prev["AMT_ANNUITY"] / prev["AMT_CREDIT"]
    
    #FEATURE 4 - "NEW_ANN/CDT" | tam tutarın vadeye bölümü
    prev["NEW_CDT/PAY"] = prev["AMT_CREDIT"] / prev["CNT_PAYMENT"]
    
    # FEATURE 5 - "NEW_ABILITY" | müşteri geçmiş aylık kredi tutarının maaşına oranı
    prev["NEW_PAY_ABILITY_PERC"] = prev["NEW_CDT/PAY"] / prev["AMT_ANNUITY"] 
    
    # FEATURE 6 - "NEW_ABILITY" | peşinat / maaş
    prev["NEW_PAY_ANN/DOWN_PERC"] = prev["AMT_ANNUITY"] / prev["AMT_DOWN_PAYMENT"] 
    
    # Previous applications numeric features
    num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
        'NEW_APP_CREDIT_PERC': ['min', 'max', 'mean'],
        'NEW_CREDIBILITY': ["sum","mean"],
        'NEW_ANN/CDT_PERC': ['min', 'max', 'mean'],
        'NEW_CDT/PAY': ['min', 'max', 'mean'],
        'NEW_PAY_ABILITY_PERC': ['min', 'max', 'mean'],
        'NEW_PAY_ANN/DOWN_PERC': ['min', 'max', 'mean']
    }
    prev, cat_cols = one_hot_encoder(prev, nan_as_category= True)
    # Previous applications categorical features
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean']
    
    prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
    # Previous Applications: Approved Applications - only numerical features
    approved = prev[prev["NAME_CONTRACT_STATUS"] == 0]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')
    # Previous Applications: Refused Applications - only numerical features
    refused = prev[prev["NAME_CONTRACT_STATUS"] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
    refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
    prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')
    del refused, refused_agg, approved, approved_agg, prev
    gc.collect()
    return prev_agg

In [5]:
def pos_cash(num_rows = None, nan_as_category = True):
    pos = pd.read_csv(r'C:\Users\oe\Desktop\HomeCreditDefaultRisk\data\POS_CASH_balance.csv', nrows = num_rows)
    pos, cat_cols = one_hot_encoder(pos, nan_as_category= True)
    # Features
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size'],
        'SK_DPD': ['max', 'mean'],
        'SK_DPD_DEF': ['max', 'mean']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    
    pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
    pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
    # Count pos cash accounts
    pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()
    del pos
    gc.collect()
    return pos_agg
    
# Preprocess installments_payments.csv
def installments_payments(num_rows = None, nan_as_category = True):
    ins = pd.read_csv(r'C:\Users\oe\Desktop\HomeCreditDefaultRisk\data\installments_payments.csv', nrows = num_rows)
    ins, cat_cols = one_hot_encoder(ins, nan_as_category= True)
    # Percentage and difference paid in each installment (amount paid and installment value)
    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
    # Days past due and days before due (no negative values)
    ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
    ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)
    # Features: Perform aggregations
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DPD': ['max', 'mean', 'sum'],
        'DBD': ['max', 'mean', 'sum'],
        'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
    # Count installments accounts
    ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()
    del ins
    gc.collect()
    return ins_agg

# Preprocess credit_card_balance.csv
def credit_card_balance(num_rows = None, nan_as_category = True):
    cc = pd.read_csv(r'C:\Users\oe\Desktop\HomeCreditDefaultRisk\data\credit_card_balance.csv', nrows = num_rows)
    cc, cat_cols = one_hot_encoder(cc, nan_as_category= True)
    # General aggregations
    cc.drop(['SK_ID_PREV'], axis= 1, inplace = True)
    cc_agg = cc.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])
    # Count credit card lines
    cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()
    del cc
    gc.collect()
    return cc_agg

In [6]:
# LightGBM GBDT with KFold or Stratified KFold
# Parameters from Tilii kernel: https://www.kaggle.com/tilii7/olivier-lightgbm-parameters-by-bayesian-opt/code
def kfold_lightgbm(df, num_folds, stratified = False, debug= False):
    # Divide in training/validation and test data
    train_df = df[df['TARGET'].notnull()]
    test_df = df[df['TARGET'].isnull()]
    print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
    del df
    gc.collect()
    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=1001)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=1001)
    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]

        # LightGBM parameters found by Bayesian optimization
        clf = LGBMClassifier(
            nthread=6,
            n_estimators=10000,
            learning_rate=0.02,
            num_leaves=34,
            colsample_bytree=0.9497036,
            subsample=0.8715623,
            max_depth=8,
            reg_alpha=0.041545473,
            reg_lambda=0.0735294,
            min_split_gain=0.0222415,
            min_child_weight=39.3259775,
            silent=-1,
            verbose=-1, )

        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
            eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)

        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
        sub_preds += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()

    print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))
    # Write submission file and plot feature importance
    if not debug:
        test_df['TARGET'] = sub_preds
        test_df[['SK_ID_CURR', 'TARGET']].to_csv(submission_file_name, index= False)
    display_importances(feature_importance_df)
    return feature_importance_df

# Display/plot feature importance
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances01.png')

In [ ]:
def main(debug = False):
    num_rows = 10000 if debug else None
    df = application_train_test(num_rows)
    with timer("Process bureau and bureau_balance"):
        bureau = bureau_and_balance(num_rows)
        print("Bureau df shape:", bureau.shape)
        df = df.join(bureau, how='left', on='SK_ID_CURR')
        del bureau
        gc.collect()
    with timer("Process previous_applications"):
        prev = previous_applications(num_rows)
        print("Previous applications df shape:", prev.shape)
        df = df.join(prev, how='left', on='SK_ID_CURR')
        del prev
        gc.collect()
    with timer("Process POS-CASH balance"):
        pos = pos_cash(num_rows)
        print("Pos-cash balance df shape:", pos.shape)
        df = df.join(pos, how='left', on='SK_ID_CURR')
        del pos
        gc.collect()
    with timer("Process installments payments"):
        ins = installments_payments(num_rows)
        print("Installments payments df shape:", ins.shape)
        df = df.join(ins, how='left', on='SK_ID_CURR')
        del ins
        gc.collect()
    with timer("Process credit card balance"):
        cc = credit_card_balance(num_rows)
        print("Credit card balance df shape:", cc.shape)
        df = df.join(cc, how='left', on='SK_ID_CURR')
        del cc
        gc.collect()
    with timer("Run LightGBM with kfold"):
        feat_importance = kfold_lightgbm(df, num_folds= 10, stratified= False, debug= debug)

if __name__ == "__main__":
    submission_file_name = "submission_kernel02.csv"
    with timer("Full model run"):
        main()

Train samples: 307511, test samples: 48744
Grouping and Sorting done
Difference days calculated
Difference in Dates between Previous CB applications is CALCULATED 


<ipython-input-3-5b9cbb74052e>:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  B1['DAYS_CREDIT_ENDDATE1'] = B1['DAYS_CREDIT_ENDDATE']


Grouping and Sorting done
Difference days calculated
Bureau df shape: (305811, 105)
Process bureau and bureau_balance - done in 608s
Previous applications df shape: (338857, 240)
Process previous_applications - done in 20s
Pos-cash balance df shape: (337252, 18)
Process POS-CASH balance - done in 9s
Installments payments df shape: (339587, 26)
Process installments payments - done in 27s
Credit card balance df shape: (103558, 141)
Process credit card balance - done in 14s
Starting LightGBM. Train shape: (307507, 655), test shape: (48744, 655)
Training until validation scores don't improve for 200 rounds
[200]	training's auc: 0.796388	training's binary_logloss: 0.234189	valid_1's auc: 0.778546	valid_1's binary_logloss: 0.245972
[400]	training's auc: 0.819046	training's binary_logloss: 0.224912	valid_1's auc: 0.788106	valid_1's binary_logloss: 0.242402
[600]	training's auc: 0.834288	training's binary_logloss: 0.21881	valid_1's auc: 0.791237	valid_1's binary_logloss: 0.241359
[800]	trainin